In [1]:
#imports
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import glob as glob

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown


from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file


In [ ]:
#For HLR
# file = r"C:\Users\clutz\Downloads\EPB_HLR_On-Site_Tracker_OnSite_Tracker_HLR_1743598638.xlsx"
# data = pd.read_excel(file)
# state = 'NC'

# bootcamp_included = True
# data

: 

In [ ]:
#FOR ECLS   
file = r"c:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\data drop\ECLS_CC_ELN_24_-_Attendance_Tracker_ECLS_2024_Attendance_Tracking_1743623133.xlsx"
data = pd.read_excel(file)
# state = 'OK'

# bootcamp_included = False
data

,ECLS | CC | ELN 24 - Attendance Tracker,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86
0,ECLS 2024 Attendance Tracking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name,State,Attendee Type,Registration Status,Registration Date,CC Attendee,ELN Attendee,Salutation,Title,Organization,...,Flight DepartureDate,Prep Call Status,Date of Prep Call,Assigned Vechicle #,Arrival P/U Time,Departure P/U Time,Crosscheck Complete,Crosscheck Status,Arrival Status,Departure Vechicle Type
2,"Rittling, Sarah",District of Columbia,Partner,Checked in,2024-06-03 00:00:00,NaN,NaN,NaN,Executive Director,FFYF,...,2024-09-06 00:00:00,Scheduling,2024-08-22 00:00:00,NaN,NaN,NaN,Check Complete,NaN,NaN,Black Car
3,"Olejnik, Adrienne",Kansas,Agency,Checked in,NaN,NaN,NaN,NaN,Vice President,Kansas Action for Children,...,2024-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,Check Complete,NaN,NaN,NaN
4,"Gunderia, Sunil",California,Funder,Registered,2024-08-26 00:00:00,NaN,NaN,NaN,Chief Innovation Officer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Check Complete,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,"Stoffel, Julie",Wisconsin,Agency,Checked in,2024-08-27 00:00:00,NaN,NaN,NaN,Director of Community Outreach and Engagement,Wisconsin Early Childhood Association,...,2024-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,Check Complete,NaN,NaN,NaN
905,"Stubbs, Shelia",Wisconson,Legislator,Invited,NaN,NaN,NaN,Representative,Representative,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906,"Oman, Alan",Michigan,Agency,Checked in,2024-08-25 00:00:00,NaN,NaN,NaN,Senior Advisor,NaN,...,2024-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,Check Complete,NaN,NaN,NaN
907,"Brant, Lachelle",Wyoming,Governor's Office,Checked in,2024-08-03 00:00:00,NaN,NaN,NaN,Senior Policy Advisor,NaN,...,2024-09-06 00:00:00,NaN,NaN,NaN,NaN,NaN,Check Complete,NaN,NaN,Shuttle


: 

In [ ]:
# #FOR OKLAHOMA
# file = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\State Engagement\Oklahoma\Jan 2025 SLR\Events-Planning and Logistics\OKC_On-Site_Tracker_OKC-FINAL.xlsx"
# data = pd.read_excel(file)
# state = 'OK'

# bootcamp_included = False
# data

: 

In [ ]:
def clean_rows(df, l_col_pat):
    #get start row and cut df
    for row in df.itertuples(index=True):
        # print(len(row))
        # print( type(row))
        # print(row.values)
        row_string = "|".join(map(str, row))
        # print(row_string)

        if re.search(r'Registration Status', row_string):
            # print(row.Index)
            # print(row_string)
            start_index = row.Index
            continue


    #implement row narrowing
    event_data_cl = df.iloc[start_index:,:].reset_index(drop=True)
    event_data_cl.columns = event_data_cl.iloc[0]
    event_data_cl = event_data_cl.iloc[1:,:]


    #get col subset
    for name,value in event_data_cl.items():
        print(name)
        if re.search(l_col_pat, str(name)):
            stop_col = name
            print(f'stop_col: {stop_col}')
            break

    #implement col narrowing
    attendance_tracking_df = event_data_cl.loc[:,:stop_col]
    return attendance_tracking_df
# attendance_tracking_df
# print(event_data_cl.columns)
# print(event_data.head(2).to_string())


: 

In [ ]:
df = clean_rows(data, r'Organization' )
df

Name
State
Attendee Type
Registration Status
Registration Date
CC Attendee
ELN Attendee
Salutation
Title
Organization
stop_col: Organization


,Name,State,Attendee Type,Registration Status,Registration Date,CC Attendee,ELN Attendee,Salutation,Title,Organization
1,"Rittling, Sarah",District of Columbia,Partner,Checked in,2024-06-03 00:00:00,NaN,NaN,NaN,Executive Director,FFYF
2,"Olejnik, Adrienne",Kansas,Agency,Checked in,NaN,NaN,NaN,NaN,Vice President,Kansas Action for Children
3,"Gunderia, Sunil",California,Funder,Registered,2024-08-26 00:00:00,NaN,NaN,NaN,Chief Innovation Officer,NaN
4,"DiMario, Alana",Rhode Island,Legislator,Checked in,2024-07-29 00:00:00,Yes,NaN,Senator,Senator,NaN
5,"Fairchild, Mark",Indiana,State,Registered,2024-08-15 00:00:00,NaN,NaN,NaN,Executive Director,Commission on Improving the Status of Children
...,...,...,...,...,...,...,...,...,...,...
903,"Stoffel, Julie",Wisconsin,Agency,Checked in,2024-08-27 00:00:00,NaN,NaN,NaN,Director of Community Outreach and Engagement,Wisconsin Early Childhood Association
904,"Stubbs, Shelia",Wisconson,Legislator,Invited,NaN,NaN,NaN,Representative,Representative,NaN
905,"Oman, Alan",Michigan,Agency,Checked in,2024-08-25 00:00:00,NaN,NaN,NaN,Senior Advisor,NaN
906,"Brant, Lachelle",Wyoming,Governor's Office,Checked in,2024-08-03 00:00:00,NaN,NaN,NaN,Senior Policy Advisor,NaN


: 

In [ ]:

def split_name(name):
    # Check if the name is in "Last, First" format
    if "," in name:
        last, first = map(str.strip, name.split(",", 1))
    else:
        # General pattern for "First Last" handling apostrophes and optional suffixes
        pattern = re.compile(
            r'^(?P<first>.*?)\s+(?P<last>(?:(?:La|Du|De)\s+[\w\']+)|[\w\']+)\s*(?:Jr\.?|Sr\.?|II|III|IV)?\s*$',
            re.UNICODE
        )
        match = pattern.search(name)
        if match:
            first, last = match.group('first').strip(), match.group('last').strip()
        else:
            print(f"No match for: {name}")
            return None, None
    
    # Remove any lingering commas
    return first.replace(',', '').strip(), last.replace(',', '').strip()

: 

In [ ]:
# get first and last name
df[['last_name', 'first_name']] = df['Name'].apply(
    lambda x: pd.Series(split_name(x)) if isinstance(x, str) else pd.Series([None, None])
)
df.columns

Index(['Name', 'State', 'Attendee Type', 'Registration Status',
       'Registration Date', 'CC Attendee', 'ELN Attendee', 'Salutation',
       'Title', 'Organization', 'last_name', 'first_name'],
      dtype='object', name=0)

: 

In [ ]:
#pull in template and get col mapping
template = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\attendance\attendance data\to be processed\2025_GAR_attendance.xlsx"
template_df = pd.read_excel(template)
print(template_df.columns)
col_mapping = {}
#go through template
mapped = []
for col in template_df.columns:
    #get col
    search_string = col.replace('_name','').lower()
    print('__________________')
    print(f'search string: {search_string}')


    #look for col that matchs
    #for each col look for it in the merged_df cols
    not_mapped = []
    cols = df.columns
    # print(type(cols))
    for m in cols:
        #return found value
        if m in mapped:
            continue
        # print(f'${m}')
        alter_m = m.replace('_name','').lower()
        if re.search(rf'{alter_m}', search_string):
            print(f'r1: mapping {m} to {col} (alter_m: {alter_m})')
            col_mapping[m] = col
            mapped.append(m)
            # found_col = True
            break
        elif re.search(r'(?:[Hh]onorific|[Ss]alutation)', search_string) and re.search(r'(?:[Hh]onorific|[Ss]alutation)', alter_m):
            print(f'r1.5: mapping {m} to {col}(variables: {m}/{search_string})')
            col_mapping[m] = col
            break
        #if column has role or type in it 
        elif re.search(r'\b[Tt]ype\b|\b[Rr]ole\b', search_string) and re.search(r'\b[Tt]ype\b|\b[Rr]ole\b', m):
            # match = re.search(r'\b[Tt]ype\b|\b[Rr]ole\b', search_string)
            # print(match)
            # print(f"r2: {match}")

            print(f'r2: mapping {m} to {col}(variables: {m}/{search_string})')
            col_mapping[m] = col
            mapped.append(m)
            # found_col = True
            break
        elif re.search(r'^[Oo]rg', search_string) and re.search(r'^[Oo]rg', str(m)):
            print(f'r3: mapping {m} to {col} (variables: {str(m)}{search_string})')
            col_mapping[m] = col
            mapped.append(m)
            # found_col = True
            break
        else:
            not_mapped.append(m)
            # print(f'tried mapping {m} from {search_string}')
    

    # print('not_mapped')
    # print(not_mapped)
        

            
# list

#print out of unassigned and unaccessed
#not assigned
n_assigned_to_dict = [x for x in template_df.columns if x not in col_mapping.values()]

#not pulled
n_accessed = [x for x in df.columns if x not in col_mapping.keys()]


Index(['first_name', 'last_name', 'honorific', 'title', 'org', 'district',
       'role', 'state', 'short_convening'],
      dtype='object')
__________________
search string: first
r1: mapping first_name to first_name (alter_m: first)
__________________
search string: last
r1: mapping last_name to last_name (alter_m: last)
__________________
search string: honorific
r1.5: mapping Salutation to honorific(variables: Salutation/honorific)
__________________
search string: title
r1: mapping Title to title (alter_m: title)
__________________
search string: org
r3: mapping Organization to org (variables: Organizationorg)
__________________
search string: district
__________________
search string: role
r2: mapping Attendee Type to role(variables: Attendee Type/role)
__________________
search string: state
r1: mapping State to state (alter_m: state)
__________________
search string: short_convening


: 

In [ ]:

print(f'not assigned: {n_assigned_to_dict}')
print(f'not accessed: {n_accessed}')



not assigned: ['district', 'short_convening']
not accessed: ['Name', 'Registration Status', 'Registration Date', 'CC Attendee', 'ELN Attendee']


: 

In [ ]:
#ONLY RUN IF YOU ARE DONE: assigns np.nan to any columns not included per template
for n in n_assigned_to_dict:
    if n == 'short_convening':
        df[n] = False
    else:
        df[n] = np.nan




        


: 

In [ ]:
#remapping most names
df_remapped = df.rename(columns=col_mapping)


df_remapped = df_remapped[~df_remapped['role'].str.contains('THI Staff', na = False)]
ecls_df = df_remapped[df_remapped['Registration Status'].str.contains(r'^[Cc]hecked in', regex = True, na = False)].reset_index(drop = True)
cc_df = ecls_df[ecls_df['CC Attendee'].str.contains('yes', case=False, na=False)].reset_index(drop = True)
eln_df = ecls_df[ecls_df['ELN Attendee'].str.contains('yes', case=False, na=False)].reset_index(drop = True)
all_dfs = [ecls_df, cc_df, eln_df]
df_names = ['ECLS', 'CC', 'ELN']
new_dict = dict(zip(df_names,all_dfs))




: 

In [ ]:
# for k,v in new_dict.items():
#     if 'ECLS' in str(k):
#         continue
#     print('__________________________')
#     print(k)
#     print(type(v))
#     print(v.to_string())
#     print(v.shape[0])

: 

In [ ]:
cols_to_ext = []
cols_to_ext.extend(list(template_df.columns))
event_cols = []
for n in n_accessed:
    print(n)
    if re.search(r'^[CELN]{2,3}\s?Attendee', str(n)):
        cols_to_ext.append(n)
        event_cols.append(n)
    
print(cols_to_ext)

Name
Registration Status
Registration Date
CC Attendee
ELN Attendee
['first_name', 'last_name', 'honorific', 'title', 'org', 'district', 'role', 'state', 'short_convening', 'CC Attendee', 'ELN Attendee']


: 

In [ ]:
print(event_cols)
print(cols_to_ext)
# for l in event_cols:
#     if l in cols_to_ext:
#         cols_to_ext.remove(col)

final_ext_cols = [col for col in cols_to_ext if col not in event_cols]
final_ext_cols


['CC Attendee', 'ELN Attendee']
['first_name', 'last_name', 'honorific', 'title', 'org', 'district', 'role', 'state', 'short_convening', 'CC Attendee', 'ELN Attendee']


['first_name',
 'last_name',
 'honorific',
 'title',
 'org',
 'district',
 'role',
 'state',
 'short_convening']

: 

In [ ]:
export_dict = {}
for k,v in new_dict.items():
    #remap column names
    new_df = v.loc[:,final_ext_cols]
    # ecls_df.columns
    export_dict[k] = new_df
# print(all_dfs)

: 

________________
ECLS
0   first_name last_name       honorific  \
0     Rittling     Sarah             NaN   
1      Olejnik  Adrienne             NaN   
2      DiMario     Alana         Senator   
3      Fortman       Dan             NaN   
4      Marzola    Elaine  Assemblymember   
..         ...       ...             ...   
358    Farrell       Jim             Mr.   
359    Carlson  Jennifer  Representative   
360    Stoffel     Julie             NaN   
361       Oman      Alan             NaN   
362      Brant  Lachelle             NaN   

0                                            title  \
0                               Executive Director   
1                                   Vice President   
2                                          Senator   
3                                              NaN   
4                                   Assemblymember   
..                                             ...   
358                        Communications Director   
359              

: 

In [ ]:
#Event Subsets

: 

In [ ]:
#other variation that looks for 'checked in' for relevant columns
# event_sub_dfs = {}
# for e in event_cols:
#     ext_df = ecls_df[ecls_df[e].str.contains(r'^[Cc]hecked', regex = True, na = False)].reset_index(drop=True)
#     ext_df = ext_df.drop(event_cols, axis = 1)
#     event_sub_dfs[e] = ext_df
    # print(ext_df.head(2).to_string())


: 

In [ ]:
#pull out sub events and aggregate total for over 2 days of main event
# df_list = list(event_sub_dfs.values())
# print(len(df_list))

0


: 

In [ ]:

# hlr_dfs = df_list[1:3]
# main_slr = pd.concat(df_list)
# main_slr = main_slr.drop_duplicates().sort_values('last_name')


# bootcamp = df_list[0]


# print(len(main_hlr))
# dupes = main_hlr[main_hlr.duplicated(keep='first')].sort_values('last_name')
# print(len(main_hlr_cl))
# print(main_hlr_cl.to_string())


: 

In [ ]:
for k,v in export_dict.items():
    os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\attendance\attendance data\output')
    #main file export
    if k == 'ECLS':
        filename = k+"attendance.xlsx"
    else:
        filename = f'ECLS_{k}.xlsx'
    
    v.to_excel(filename, index=False)
    #bootcamp file
    
